In [ ]:
from getpass import getpass

DASHSCOPE_API_KEY = getpass()

import os
os.environ["DASHSCOPE_API_KEY"] = "sk-cbcc1fb859b1456885a270eecbec6369"


In [159]:
database_config = {
    "host": "gz-cdb-5scrcjb5.sql.tencentcdb.com",
    "user": "buildsensePublic",
    "password": "buildsense905",
    "database": "sele",
    "port": 63432
}


from sqlalchemy import create_engine
# 构建数据库连接URL
db_url = f"mysql+mysqlconnector://{database_config['user']}:{database_config['password']}@{database_config['host']}:{database_config['port']}/{database_config['database']}"
# 创建SQLAlchemy引擎
engine = create_engine(db_url)

from langchain_community.utilities.sql_database import SQLDatabase

# 连接数据库
sql_database = SQLDatabase(engine)

# from langchain_openai.chat_models.base import ChatOpenAI

# llm = ChatOpenAI(
#     temperature=0.1,
#     openai_api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
#     openai_api_key="sk-cbcc1fb859b1456885a270eecbec6369",
#     model_name="qwen-turbo"
# )


from langchain_community.llms import Tongyi
llm = Tongyi(
    model_name="qwen-turbo", 
    temperature=0.2,
    dashscope_api_key="sk-cbcc1fb859b1456885a270eecbec6369"
)


from langchain_community.agent_toolkits import create_sql_agent
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

# 创建SQLDatabaseToolkit
toolkit = SQLDatabaseToolkit(llm=llm, db=sql_database)


# 创建SQL Agent ,openai-tools
sql_agent = create_sql_agent(llm, toolkit, agent_type="zero-shot-react-description", verbose=True)



In [ ]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotPromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
    SystemMessagePromptTemplate,
)

prefix = """
您是一位旨在与SQL数据库交互的代理。

面对输入的问题,创建一个语法正确的MySQL查询以执行。
查看查询结果,并返回答案。
除非用户指定了希望获取的具体样本数量,否则查询结果限制为最多10条。
您可以根据相关列对结果进行排序,以返回数据库中最有趣的示例。
不要从特定表中查询所有列,仅根据问题需求请求相关列。
您可以使用与数据库交互的工具,且仅限于使用这些工具。
构建最终答案时,仅依据这些工具返回的信息。
执行查询前,您必须仔细检查您的查询语句。如果查询时遇到错误,重新编写查询并重试。
切勿对数据库做出任何DML操作(如INSERT, UPDATE, DELETE, DROP等)。
如果问题似乎与数据库无关,则直接回答"我不知道"。
"""

suffix = """
使用以下格式：

问题：你需要回答的输入问题
思考：你应该总是思考接下来该做什么
行动：要采取的行动，应该是以下工具之一 [{tool_names}]
行动输入：对行动的输入
观察：行动的结果
...（这个思考/行动/行动输入/观察可以重复N次）
思考：我现在知道了最终的答案
最终答案：对原始输入问题的最终回答

开始！

问题：{input}
思考：{agent_scratchpad}'

"""

example_prompt = PromptTemplate(
    input_variables=["input", "query"],
    template="Human: {input}\nAction: sql_query\nAction Input: {query}",
)

few_shot_prompt = FewShotPromptTemplate(
    prefix=prefix,
    suffix=suffix,
    example_prompt=example_prompt,
    examples=[
        {"input": "列出所有艺术家", "query": "SELECT * FROM Artist;"},
        {"input": "找到艺术家'AC/DC'的所有专辑", "query": "SELECT * FROM Album WHERE ArtistId = (SELECT ArtistId FROM Artist WHERE Name = 'AC/DC');"},
    ],
    input_variables=["input"],
)


In [170]:
full_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=few_shot_prompt),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)



In [172]:
# Example formatted prompt
prompt_val = full_prompt.invoke(
    {
        "input": "How many arists are there",
        "tool_names": ["sql_query"],
        "top_k": 5,
        "dialect": "SQLite",
        "agent_scratchpad": [],
    }
)
print(prompt_val.to_string())

System: 
您是一位旨在与SQL数据库交互的代理。

面对输入的问题,创建一个语法正确的MySQL查询以执行。
查看查询结果,并返回答案。
除非用户指定了希望获取的具体样本数量,否则查询结果限制为最多10条。
您可以根据相关列对结果进行排序,以返回数据库中最有趣的示例。
不要从特定表中查询所有列,仅根据问题需求请求相关列。
您可以使用与数据库交互的工具,且仅限于使用这些工具。
构建最终答案时,仅依据这些工具返回的信息。
执行查询前,您必须仔细检查您的查询语句。如果查询时遇到错误,重新编写查询并重试。
切勿对数据库做出任何DML操作(如INSERT, UPDATE, DELETE, DROP等)。
如果问题似乎与数据库无关,则直接回答"我不知道"。


Human: 列出所有艺术家
Action: sql_query
Action Input: SELECT * FROM Artist;

Human: 找到艺术家'AC/DC'的所有专辑
Action: sql_query
Action Input: SELECT * FROM Album WHERE ArtistId = (SELECT ArtistId FROM Artist WHERE Name = 'AC/DC');


使用以下格式：

问题：你需要回答的输入问题
思考：你应该总是思考接下来该做什么
行动：要采取的行动，应该是以下工具之一 [['sql_query']]
行动输入：对行动的输入
观察：行动的结果
...（这个思考/行动/行动输入/观察可以重复N次）
思考：我现在知道了最终的答案
最终答案：对原始输入问题的最终回答

开始！

问题：How many arists are there
思考：[]'


Human: How many arists are there


In [ ]:
# 使用Agent回答问题
result = sql_agent.invoke("价格最高的十个招标公告")


In [131]:

from langchain_community.llms import Tongyi
llm = Tongyi(temperature=0.1)


In [124]:
llm.invoke("What NFL team won the Super Bowl in the year Justin Bieber was born?")



'Justin Bieber was born on March 1, 1994. The Super Bowl that year, Super Bowl XXVIII, was won by the Dallas Cowboys. They defeated the Buffalo Bills with a score of 30-13.'

In [115]:
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI

import os

# 设置 Tavily API 密钥
os.environ["TAVILY_API_KEY"] = "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"

# export TAVILY_API_KEY={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}
tools = [TavilySearchResults(max_results=1)]
prompt = hub.pull("hwchase17/openai-tools-agent")



agent = create_tool_calling_agent(llm, tools, prompt)



In [116]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)



In [ ]:
from langchain_core.messages import AIMessage, HumanMessage

agent_executor.invoke(
    {
        "input": "what's my name? Don't use tools to look this up unless you NEED to",
        "chat_history": [
            HumanMessage(content="hi! my name is bob"),
            AIMessage(content="Hello Bob! How can I assist you today?"),
        ],
    }
)




In [175]:
from config import api_key_qwen
from aiModel import QwenModel
from ai import call_model


def table_sum_agent(structure,samples):
    tools = [
        {
        "type": "function",
        "function": {
            "name": "save_summary",
            "description": "将总结信息保存到数据库中。包对对整个表格以及每一列字段的总结。",
            "parameters": {
                "type": "object",
                "properties": {
                    "table_summary": {
                        "description": "对表格功能用户的一句话总结",
                        "type": "string"
                    },
                        "column_summaries": {
                        "description": """
                        对每一个字段的一个总结，{
                            "column1_name": "Summary for column 1 based on the samples and structure.",
                            "column2_name": "Summary for column 2 based on the samples and structure."}
                            """,
                        "type": "dict"
                    }
                },
                "required": ["table_summary"]
            },
        },
        }
    ]

    role_setting = '''
    Prompt: "根据表格的结构和样本条目，总结表格的整体用途和每列存储的信息类型。为表格的总体用途生成一句详尽的总结，并为每一列创建一个详尽的总结字典。使用'save_summary'函数保存这些总结。"


任务:
1. 分析表格结构，确定其预定功能。
2. 查看样本条目，了解每一列存储的数据类型及其对表格功能的贡献。
3. 生成表格目的的详尽单句总结。
4. 创建一个字典，为每一列生成总结，说明基于结构和样本的信息类型。
5. 使用'save_summary'工具将生成的总结保存到数据库中。

工具:
- save_summary: 将生成的总结保存到数据库中的函数。该函数需要表格的一句话总结和每一列的总结字典。

预期输出:
- 表格整体的一句话总结。
- 包含每列总结的字典。
- 成功执行'save_summary'函数以保存这些总结。

确保总结的准确性和相关性以符合所提供的结构和样本数据，并遵守使用'save_summary'函数的所需格式和详细要求。

      '''
    messages = [{"role": "system","content": role_setting}]
    messages.append({"role": "user","content": '数据表结构是：' + structure})
    messages.append({"role": "user","content": '表格样本条目是：' + samples})
    
    model_name = "qwen-plus" 
    response = QwenModel(api_key=api_key_qwen, model=model_name, temperature=0.2,tools=tools)
    total_usage = 0
    call_model(response, messages)
    messages.append(response.message_to_append)
    total_usage += response.total_tokens

    max_loop_count = 10
    loop_count = 0
    while loop_count < max_loop_count and response.tool_calls:
        args = response.function_args

        if response.function_name == "save_summary":
            try:
                # function_result = save_summary(**json.loads(args))
                return args
            except Exception as e:
                function_result = {"error": str(e)}

        messages.append({
            "role": "tool",
            "content": f"{function_result}",
            "tool_call_id":response.tool_calls['id']
        })
        
        call_model(response, messages)
        total_usage += response.total_tokens
        messages.append(response.message_to_append)
    return response.content


In [176]:
import mysql.connector
import json

# 建立数据库连接
connection = mysql.connector.connect(
    host="gz-cdb-5scrcjb5.sql.tencentcdb.com",
    user="db",
    password="dbdb905905",
    database="sele",
    port=63432
)

def get_tables(cursor):
    cursor.execute("SHOW TABLES")
    return [table[0] for table in cursor.fetchall()]

def get_table_structure(cursor, table_name):
    cursor.execute(f"SHOW CREATE TABLE {table_name}")
    return cursor.fetchone()[1]

def get_sample_data(cursor, table_name):
    cursor.execute(f"SELECT * FROM {table_name} LIMIT 5")
    rows = cursor.fetchall()
    # 将行数据转换成字符串，以便传递给AI分析器
    sample_str = "\n".join([str(row) for row in rows])
    return sample_str



In [177]:
cursor = connection.cursor()
tables = get_tables(cursor)
results = []


In [184]:
tables[12]

'tender_key_detail'

In [185]:
structure = get_table_structure(cursor, tables[12])
samples = get_sample_data(cursor, tables[12])
     

In [188]:
result = table_sum_agent(structure,samples)

In [198]:
def summary_formatter(table_summary,column_summaries):
    return table_summary,column_summaries

In [193]:
json.loads(result)

{'table_summary': 'tender_key_detail表格用于存储招标项目的关键详细信息，包括投标价格、施工期限、施工面积、建设成本以及承包商的资质、资格等级、专业领域、项目经验等要求。',
 'column_summaries': {'tender_id': '招标项目的唯一标识符',
  'bid_price': '投标的价格',
  'construction_duration': '施工的预计持续时间（单位：天）',
  'construction_area': '施工区域的面积（可能为浮点数，表示不精确值）',
  'construction_cost': '施工的总成本',
  'qualification_type': '承包商需要具备的工程类型资质',
  'qualification_level': '承包商的资质等级',
  'qualification_profession': '承包商需要具备的专业领域',
  'title_level': '项目负责人的职称',
  'education': '项目负责人所需的学历',
  'performance_requirements': '承包商以往的业绩要求',
  'simultaneous_projects_limit': '承包商可同时承担的项目限制',
  'qualification_profession_addition': '额外的资质或专业要求'}}

In [197]:
summary_formatter(**json.loads(result))

('tender_key_detail表格用于存储招标项目的关键详细信息，包括投标价格、施工期限、施工面积、建设成本以及承包商的资质、资格等级、专业领域、项目经验等要求。',
 {'tender_id': '招标项目的唯一标识符',
  'bid_price': '投标的价格',
  'construction_duration': '施工的预计持续时间（单位：天）',
  'construction_area': '施工区域的面积（可能为浮点数，表示不精确值）',
  'construction_cost': '施工的总成本',
  'qualification_type': '承包商需要具备的工程类型资质',
  'qualification_level': '承包商的资质等级',
  'qualification_profession': '承包商需要具备的专业领域',
  'title_level': '项目负责人的职称',
  'education': '项目负责人所需的学历',
  'performance_requirements': '承包商以往的业绩要求',
  'simultaneous_projects_limit': '承包商可同时承担的项目限制',
  'qualification_profession_addition': '额外的资质或专业要求'})

In [199]:


def main():
    cursor = connection.cursor()
    tables = get_tables(cursor)
    results = []

    for table in tables:
        structure = get_table_structure(cursor, table)
        samples = get_sample_data(cursor, table)
        response = table_sum_agent(structure, samples)
        table_summary, columns_summary = summary_formatter(**json.loads(response))
        results.append({
            "table": table,
            "summary": table_summary,
            "columns": columns_summary
        })

    cursor.close()
    connection.close()

    # 将结果保存为JSON文件
    with open('database_summary.json', 'w') as f:
        json.dump(results, f, indent=4, ensure_ascii=False)

if __name__ == "__main__":
    main()


In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
import json

# 假设已经有一个包含数据库所有表及其字段摘要的JSON文件
with open('database_summary.json', 'r') as f:
    database_summary = json.load(f)

# 提取字段总结和对应的字段名
field_summaries = []
metadata_list = []

for table in database_summary:
    # 对表级总结的处理
    field_summaries.append(table["summary"])
    metadata_list.append({"field_name": "", "table_name": table["table"]})

    # 对字段级总结的处理
    for column_name, column_info in table["columns"].items():
        field_summaries.append(column_info["summary"])
        metadata_list.append({"field_name": column_name, "table_name": table["table"]})

# 初始化嵌入模型
embeddings = OpenAIEmbeddings()

# 对每个总结进行嵌入
vecs = embeddings.embed_documents(field_summaries)

# 初始化向量存储,并添加嵌入向量和元数据
vector_store = Chroma.from_texts(field_summaries, vecs, metadatas=metadata_list)

# 搜索
query = "我需要找到存储客户地址的字段"
results = vector_store.similarity_search(query, k=2)

print(f"相关字段名:")
for result in results:
    metadata = result.metadata
    if metadata["field_name"]:  # 确保不显示表级总结
        print(f"{metadata['table_name']}.{metadata['field_name']}")  # 显示表名和字段名
